In [1]:
#导入dateframe计算工具
import pandas as pd

# #导入绘图工具
# import matplotlib.pyplot as plt
# #一般作为机器学习绘图用, 但是也能绘出更cool的图
# import seaborn as sns

#mysql 数据源工具
import sqlalchemy as db

#系统及命令行
import sys

import json
import requests

In [17]:
arguments=sys.argv
if len(arguments) <= 4:
    print("usage: {} db_host db_user db_password esHost. For example, {} 10.10.0.215:3306 root 82f12dbf497b 10.10.0.223:9200".format(arguments[0], arguments[0]))
    sys.exit(-1)

profile_db = "10.10.0.215:3306"
username = "root"
password = "82f12dbf497b"
esHost="10.10.0.223:9200"
esIndex="profile_service"
esType="extra_profile"

profile_db = sys.argv[1]
username = sys.argv[2]
password = sys.argv[3]
esHost = sys.argv[4]

print("database: {}, username: {}, password: {}, elasticsearch: {}".format(profile_db, username, password, esHost))

mysql_conn_str = 'mysql+pymysql://{}:{}@{}/profile_service?charset=utf8mb4'.format(username, password, profile_db)

user_profile_query = '''select login_id,langren_image,entrance_year from extra_profile where login_id is not null'''

usage: /ws/service/venv/py3/lib/python3.7/site-packages/ipykernel_launcher.py db_host db_user db_password esHost. For example, /ws/service/venv/py3/lib/python3.7/site-packages/ipykernel_launcher.py 10.10.0.215:3306 root 82f12dbf497b 10.10.0.223:9200
database: 10.10.0.215:3306, username: root, password: 82f12dbf497b, elasticsearch: 10.10.0.223:9200


In [18]:
try:
    engine = db.create_engine(mysql_conn_str)
    connection = engine.connect()
    df = pd.read_sql_query(user_profile_query, connection)
except:
    print("读取数据错误")
    raise "读取数据错误"
connection.close()

In [19]:
def getJsonDataFromRow(row):
    loginId = row[0]
    data = {"loginId": loginId}
    if row[1] != None:
        data["langrenImage"] = row[1]
    if row[2] != None:
        data["entranceYear"] = row[2]
    updateData = {"doc_as_upsert":True, "doc": data}
    return json.dumps(updateData)

In [20]:
for row in df.values:
    loginId = row[0]
    postUrl = "http://{}/{}/{}/{}/_update".format(esHost, esIndex, esType, loginId)
    rowData = getJsonDataFromRow(row)
    requests.post(url = postUrl, data = rowData.encode('utf-8'), headers = {"Content-Type":"application/json"})

{"doc_as_upsert": true, "doc": {"loginId": "18321593357", "langrenImage": "http://resource.zan-qian.com/test/langren_img/1543023232837.jpg", "entranceYear": "2018"}}
{"doc_as_upsert": true, "doc": {"loginId": "18260045855", "langrenImage": "http://resource.zan-qian.com/test/langren_img/1542279082976.jpg", "entranceYear": "2002"}}
{"doc_as_upsert": true, "doc": {"loginId": "18662218897", "langrenImage": "http://resource.zan-qian.com/test/langren_img/1542355923368.jpg", "entranceYear": "2017"}}
{"doc_as_upsert": true, "doc": {"loginId": "t1", "langrenImage": "http://xxx1", "entranceYear": "2018"}}
{"doc_as_upsert": true, "doc": {"loginId": "t2", "langrenImage": "http://xxx1", "entranceYear": "2018"}}
{"doc_as_upsert": true, "doc": {"loginId": "t3", "langrenImage": "http://xxx1", "entranceYear": "2018"}}
{"doc_as_upsert": true, "doc": {"loginId": "t4", "langrenImage": "http://xxx1", "entranceYear": "2018"}}
{"doc_as_upsert": true, "doc": {"loginId": "t5", "langrenImage": "http://xxx1", "e

In [21]:
# 如果image没有，则将langrenImage将其更新    
postUrl = "http://{}/{}/{}/_update_by_query?conflicts=proceed".format(esHost, esIndex, esType)
data = '''
{ 
  "query": {
    "bool": {
      "must_not": [
        {
          "exists": {
            "field": "image"
          }
        }
      ],
      "filter": {
        "exists": {
          "field": "langrenImage"
        }
      }
    }
  },
  "script": {
    "source": "ctx._source['image'] = ctx._source['langrenImage']",
    "lang": "painless"
  }
}
'''
requests.post(url = postUrl, data = data.encode('utf-8'), headers = {"Content-Type":"application/json"})
    

<Response [200]>